In [212]:
import sqlite3 
from sqlite3 import Error
import pandas as pd

> Дано 2 csv-файла: один с транзакциями за период 01.01.2023 - 20.04.2023, второй с информацией о клиентах.
> 
> **За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’**
> 
> Необходимо в Jupyter-ноутбуке выполнить следующие пункты, используя SQLite:

## Шаг 1. Подготовительный этап

> _Аналогичные действия вы делали в прошлом домашнем задании._
> 
> **(Балл - 2)** Необходимо скачать CSV-файл - <span style="color:green">transactions_for_dz2.csv</span>, создать таблицу _transaction_bd_ со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:
> 
> <br/>
> (0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции<br/>
> (1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции<br/>
> (2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента<br/>
> (3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала<br/>
> (4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции<br/>
> 
> <br/>
> Также необходимо скачать второй CSV-файл - <span style="color:green">client_info.csv</span>, создать таблицу _customer_bd_ со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:
> <br/>
> <br/>
> (0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента<br/>
> (1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте<br/>
> (2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте<br/>
> (3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента<br/>
> (4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождения клиента <br/>

Создаем функцию для создания подключения с возможностью создания БД в памяти и в файле. По-умолчанию будем использовать создание БД в памяти

In [213]:
def create_connection(in_memory=True, path=None):
    conn = None
    try:
        conn = sqlite3.connect(":memory:" if in_memory else path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return conn

Создаем функцию для выполнения запросов

In [214]:
def execute_query(query, conn):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

Создаем функцию для заполнения таблицы БД данными из датасета

In [215]:
def sql_insert(data, name, conn, m=0, n=1):
    cursor = conn.cursor()
    try:
        cursor.executemany(f'''
                INSERT INTO {name} {tuple(data.columns[m:n])} 
                    VALUES ({str("?, " * (n-m-1) + "?")})
                    ''',
                data.iloc[:, m:n].values)
        conn.commit()
        print('Data inserted successfully')
    except Error as e:
        print(f"The error '{e}' occurred")

Создаем функцию для получения информации по столбцам

In [216]:
def table_info(table_name, conn):
    c = conn.cursor()
    meta = c.execute("PRAGMA table_info('" + table_name + "')")
    for r in meta:
        print(r)

Задаем дату по-умолчанию

In [217]:
default_date = '2023-05-01'

Задаем имя таблицы БД с транзакциями

In [218]:
transaction_table = 'transaction_bd'

Задаем имя таблицы БД с клиентами

In [219]:
customer_table = 'customer_bd'

Создаем подключение

In [220]:
conn = create_connection()

Connection to SQLite DB successful


Создаем таблицу транзакций в БД.<br/>
По условию, колонки должны иметь разные типы. Посмотрим на полученный результат и типы столбцов после загрузки данных из датасета.

In [221]:
execute_query(
    f'''
    CREATE TABLE IF NOT EXISTS {transaction_table} (
        transaction_id INTEGER,
        tx_datetime NUMERIC,
        customer_id INTEGER,
        terminal_id INTEGER,
        tx_amount REAL
    );
    ''',
conn)

Query executed successfully


Загружаем данные из csv в датасет

In [222]:
trans_df = pd.read_csv('data/transactions_for_dz2.csv', delimiter=',')

Посмотрим на датасет

In [223]:
trans_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66


In [224]:
trans_df.tail()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38
1048572,1048572,2023-04-20 10:07:33,1000,5417,182.79
1048573,1048573,2023-04-20 10:07:39,3028,6439,455.44
1048574,1048574,2023-04-20 10:07:43,3252,6009,304.55


In [225]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TRANSACTION_ID  1048575 non-null  int64  
 1   TX_DATETIME     1048575 non-null  object 
 2   CUSTOMER_ID     1048575 non-null  int64  
 3   TERMINAL_ID     1048575 non-null  int64  
 4   TX_AMOUNT       1048575 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 40.0+ MB


Загружаем данные из датасета в таблицу БД

In [226]:
sql_insert(trans_df, transaction_table, conn, 0, trans_df.shape[1])

Data inserted successfully


Проверяем соответствие атрибутов условию

In [227]:
table_info(transaction_table, conn)

(0, 'transaction_id', 'INTEGER', 0, None, 0)
(1, 'tx_datetime', 'NUMERIC', 0, None, 0)
(2, 'customer_id', 'INTEGER', 0, None, 0)
(3, 'terminal_id', 'INTEGER', 0, None, 0)
(4, 'tx_amount', 'REAL', 0, None, 0)


Выберем 10 записей из таблицы и посмотрим на результат

In [228]:
pd.read_sql(
    f'''
    SELECT * 
    FROM {transaction_table} 
    LIMIT 10
    ''', 
conn)

,transaction_id,tx_datetime,customer_id,terminal_id,tx_amount
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66
5,5,2023-01-01 00:10:45,568,8803,401.17
6,6,2023-01-01 00:11:30,2803,5490,938.54
7,7,2023-01-01 00:11:44,4684,2486,206.53
8,8,2023-01-01 00:11:53,4128,8354,253.47
9,9,2023-01-01 00:13:44,541,6212,555.63


Создаем таблицу клиентов в БД.<br/>
По условию, колонки должны иметь разные типы. Посмотрим на полученный результат и типы столбцов после загрузки данных из датасета.

In [229]:
execute_query(
    f'''
    CREATE TABLE IF NOT EXISTS {customer_table} (
        client_id INTEGER,
        start_dt NUMERIC,
        end_dt NUMERIC,
        client_name TEXT,
        year_birth TEXT
    );
    ''',
conn)

Query executed successfully


Загружаем данные из csv в датасет

In [230]:
client_df = pd.read_csv('data/client_info.csv', delimiter=';')

Посмотрим на датасет

In [231]:
client_df.head()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767


In [232]:
client_df.tail()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
4983,2023-04-20,2999-12-31,Havilah,2000,4737
4984,2023-04-20,2999-12-31,Hazelyn,2000,2301
4985,2023-04-20,2999-12-31,Helaina,2000,3238
4986,2023-04-20,2999-12-31,Helene,2000,3946
4987,2023-04-20,2999-12-31,Idalis,2000,2853


In [233]:
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START_DT     4988 non-null   object
 1   END_DT       4988 non-null   object
 2   CLIENT_NAME  4988 non-null   object
 3   YEAR_BIRTH   4988 non-null   int64 
 4   CLIENT_ID    4988 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.0+ KB


Загружаем данные из датасета в таблицу БД

In [234]:
sql_insert(client_df, customer_table, conn, 0, client_df.shape[1])

Data inserted successfully


Проверяем соответствие атрибутов условию

In [235]:
table_info(customer_table, conn)

(0, 'client_id', 'INTEGER', 0, None, 0)
(1, 'start_dt', 'NUMERIC', 0, None, 0)
(2, 'end_dt', 'NUMERIC', 0, None, 0)
(3, 'client_name', 'TEXT', 0, None, 0)
(4, 'year_birth', 'TEXT', 0, None, 0)


Выберем 10 записей из таблицы и посмотрим на результат

In [236]:
pd.read_sql(
    f'''
    SELECT * 
    FROM {customer_table} 
    LIMIT 10
    ''', 
conn)

,client_id,start_dt,end_dt,client_name,year_birth
0,2213,2015-07-16,2021-01-01,Olivia,1979
1,1148,2015-07-17,2021-01-01,Emma,1979
2,2293,2015-07-18,2021-01-01,Charlotte,1979
3,1867,2015-07-19,2021-01-01,Amelia,1979
4,1767,2015-07-20,2021-01-01,Ava,1979
5,3853,2015-07-21,2021-01-01,Sophia,1979
6,1605,2015-07-22,2021-01-01,Isabella,1979
7,3273,2015-07-23,2021-01-01,Mia,1979
8,147,2015-07-24,2021-01-01,Evelyn,1979
9,1420,2015-07-25,2021-01-01,Harper,1979


## Шаг 2

> **(Балл - 2 за каждый пункт)** Написать скрипты:

> a. Вывести список всех клиентов, у которых между двумя ближайшими транзакциями был перерыв больше 35 дней хотя бы один раз (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [237]:
pd.read_sql(
    f'''
    SELECT DISTINCT 
        tmp.customer_id AS client_id,
        tmp.client_name AS client_name
    FROM (
        SELECT 
            tt.customer_id,
            ct.client_name,
            JULIANDAY(tt.tx_datetime) - LAG(JULIANDAY(tt.tx_datetime)) OVER (PARTITION BY tt.customer_id ORDER BY JULIANDAY(tt.tx_datetime)) AS diff
        FROM 
            {transaction_table} AS tt
        JOIN {customer_table} AS ct
            ON tt.customer_id = ct.client_id
            AND DATE(ct.end_dt) >= DATE('{default_date}') 
    ) AS tmp 
    WHERE tmp.diff > 35
    ''', 
conn)

,client_id,client_name
0,24,Kiarra
1,480,Vaishnavi
2,639,Malayna
3,707,Becca
4,812,Avionna
5,896,Macarena
6,1129,Kavya
7,1299,Nalah
8,1334,Blakley
9,1459,Nalia


> b. Вывести список клиентов, у которых максимальная сумма транзакции (имеется ввиду максимальное значение TX_AMOUNT) больше 50000. В ответе должен быть id клиента, максимальное значение суммы транзакций, минимальное значение суммы транзакций (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [238]:
pd.read_sql(
    f'''
    SELECT 
        tmp.customer_id AS client_id,
        tmp.max AS max_trans_amount,
        tmp.min AS min_trans_amount
    FROM (
        SELECT DISTINCT
            tt.customer_id,
            MAX(tt.tx_amount) OVER (PARTITION BY tt.customer_id) AS max,
            MIN(tt.tx_amount) OVER (PARTITION BY tt.customer_id) AS min
        FROM 
            {transaction_table} AS tt
        JOIN {customer_table} AS ct
            ON tt.customer_id = ct.client_id
            AND DATE(ct.end_dt) >= DATE('{default_date}') 
    ) AS tmp 
    WHERE tmp.max > 50000
    ''', 
conn)

,client_id,max_trans_amount,min_trans_amount
0,3013,77212.50,1.46
1,3494,53213.00,85.76
2,4253,51937.25,37.95


> c. Вывести топ-10 клиентов (сортируя их по убыванию id), у которых сумма дневных транзакций больше суммы вечерних транзакций. Условимся, что дневными считаются транзакции, которые были произведены до 12 часов, а вечерними - после 12 часов включительно. (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [239]:
pd.read_sql(
    f'''
    SELECT 
        tmp.customer_id AS client_id,
        tmp.client_name AS client_name,
        tmp.diff AS diff_btw_day_night 
    FROM (
        SELECT DISTINCT
            tt.customer_id,
            ct.client_name,
            SUM(CASE WHEN CAST(strftime('%H', tt.tx_datetime) AS INTEGER) < 12 THEN tt.tx_amount ELSE 0 END) OVER (PARTITION BY tt.customer_id) - SUM(CASE WHEN CAST(strftime('%H', tt.tx_datetime) AS INTEGER) >= 12 THEN tt.tx_amount ELSE 0 END) OVER (PARTITION BY tt.customer_id) AS diff
        FROM 
            {transaction_table} AS tt
        JOIN {customer_table} AS ct
            ON tt.customer_id = ct.client_id
            AND DATE(ct.end_dt) >= DATE('{default_date}') 
        ORDER BY diff DESC
        LIMIT 10
    ) AS tmp
    ORDER BY tmp.customer_id DESC
    ''', 
conn)

,client_id,client_name,diff_btw_day_night
0,4253,Zeina,243663.10
1,4163,Maddisyn,107707.89
2,3494,Atlee,171010.86
3,3343,Sunnie,150492.18
4,2654,Nabila,139694.44
5,2407,Avalee,97752.41
6,2032,Lyrica,128740.80
7,1918,Jovi,147364.18
8,1883,Kayci,228502.41
9,1817,Odette,140522.59


> d. Найти для каждого клиента день, когда у него была максимальная сумма транзакции, то есть вывести клиента, дату транзакции, сумму транзакции (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [240]:
pd.read_sql(
    f'''
    SELECT 
        tmp.customer_id AS client_id,
        tmp.client_name AS client_name,
        DATE(tt.tx_datetime) AS trans_date,
        tmp.max AS trans_sum
    FROM (    
        SELECT
            tt.customer_id,
            ct.client_name,
            MAX(tt.tx_amount) AS max
        FROM {transaction_table} AS tt
        JOIN {customer_table} AS ct
            ON tt.customer_id = ct.client_id
            AND DATE(ct.end_dt) >= DATE('{default_date}') 
        GROUP BY 
            tt.customer_id
    ) AS tmp 
    LEFT JOIN {transaction_table} AS tt
        ON tmp.customer_id = tt.customer_id
        AND tmp.max = tt.tx_amount
    ORDER BY tmp.customer_id
    ''', 
conn)

,client_id,client_name,trans_date,trans_sum
0,0,Eleni,2023-02-01,1258.86
1,1,Jocelyn,2023-02-07,17152.00
2,2,Bria,2023-02-21,1720.11
3,5,Zimal,2023-03-21,1156.11
4,6,Raeya,2023-03-20,369.26
...,...,...,...,...
3731,4991,Tyanna,2023-01-25,1070.26
3732,4992,Logan,2023-03-10,839.03
3733,4993,Oakland,2023-01-13,1054.57
3734,4996,Royal,2023-01-11,224.62


> e. Вычислить интегральное распределение суммы транзакций для каждого дня за весь период, то есть вывести дату транзакции, сумму транзакции и интегральное распределение по сумме (не учитываем что клиент может быть недействующим)

In [241]:
pd.read_sql(
    f'''
    SELECT 
        DATE(tt.tx_datetime) AS trans_date,
        tt.tx_amount AS trans_sum, 
        CUME_DIST() OVER(PARTITION BY tt.tx_amount ORDER BY DATE(tt.tx_datetime)) AS integ_dist_by_sum
    FROM 
        {transaction_table} AS tt
    ORDER BY
        DATE(tt.tx_datetime)
    ''', 
conn)

,trans_date,trans_sum,integ_dist_by_sum
0,2023-01-01,0.81,0.083333
1,2023-01-01,1.02,0.090909
2,2023-01-01,1.03,0.083333
3,2023-01-01,1.08,0.166667
4,2023-01-01,1.36,0.111111
...,...,...,...
1048570,2023-04-20,13024.50,1.000000
1048571,2023-04-20,18554.25,1.000000
1048572,2023-04-20,19174.75,1.000000
1048573,2023-04-20,19740.75,1.000000


Закрываем подключение

In [242]:
conn.close()